In [ ]:
import sys
sys.path.append("..")

In [ ]:
import torch
import numpy as np
import random
import torch.optim as optim

from algorithms.algorithms import *
from utils.models import *
from utils.dataloaders import *

from tabulate import tabulate

In [ ]:
device = 'cpu'
np.random.seed(0)

dataset = CreditCardClientsDataset(device=device, sensitive_feature_labels=["SEX", "AGE"])

In [ ]:
def pareto_frontier_multi(myArray):
    # Sort on first dimension
    myArray = myArray[myArray[:,0].argsort()[::-1]]
    # Add first row to pareto_frontier
    pareto_frontier = myArray[0:1,:]
    # Test next row against the last row in pareto_frontier
    for row in myArray[1:,:]:
        if row[0]<pareto_frontier[-1][0] and row[1]<pareto_frontier[-1][1]:
            # If it is better on all features add the row to pareto_frontier
            pareto_frontier = np.concatenate((pareto_frontier, [row]))
        if row[0]==pareto_frontier[-1][0] and row[1]<pareto_frontier[-1][1]:
            # If it is better on all features add the row to pareto_frontier
            pareto_frontier[-1] = row
    return pareto_frontier

def lr_kde_model_runner(dataset, hp, seeds):
    test = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}
    
    train = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}
    
    val = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}

    def append_res(l,acc,ei,dp,eo,eodd):
        l['accuracy'].append(acc)
        l['ei_disparity'].append(ei)
        l['dp_disparity'].append(dp)
        l['eo_disparity'].append(eo)
        l['eodd_disparity'].append(eodd)

    for i in range(len(seeds)):
        print('training seed', seeds[i] ,'started')
        random.seed(int(seeds[i]))
        np.random.seed(seeds[i])
        torch.manual_seed(seeds[i]) 

        model = logReg(num_features=dataset.XZ_train.shape[1])
        model = model.to(device)
        
        lr = hp['learning_rate']
        optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)
        
        results = trainer_kde_fair(
            model,
            dataset,
            optimizer,
            device,
            n_epochs=hp['n_epochs'],
            batch_size=hp['batch_size'], 
            z_blind=False,
            fairness=hp['fairness'], 
            lambda_=hp['lambda_'], 
            h=hp['h'], 
            delta_huber=hp['delta_huber'], 
            optimal_effort=True, 
            delta_effort=hp['delta_effort']
            )
        
        append_res(train,results.train_acc_hist[-1],results.train_ei_hist[-1],results.train_dp_hist[-1],results.train_eo_hist[-1],results.train_eodd_hist[-1])
        append_res(val,results.val_acc,results.val_ei,results.val_dp,results.val_eo,results.val_eodd)
        append_res(test,results.test_acc,results.test_ei,results.test_dp,results.test_eo,results.test_eodd)

    def get_res(l):
        res = {}
        res['accuracy_mean'] = np.mean(l['accuracy'])
        res['accuracy_var'] = np.std(l['accuracy'])
        res['accuracy_list'] = l['accuracy']
        res['ei_mean'] = np.mean(l['ei_disparity'])
        res['ei_var'] = np.std(l['ei_disparity'])
        res['ei_list'] = l['ei_disparity']
        res['dp_mean'] = np.mean(l['dp_disparity'])
        res['dp_var'] = np.std(l['dp_disparity'])
        res['dp_list'] = l['dp_disparity']
        res['eo_mean'] = np.mean(l['eo_disparity'])
        res['eo_var'] = np.std(l['eo_disparity'])
        res['eo_list'] = l['eo_disparity']
        res['eodd_mean'] = np.mean(l['eodd_disparity'])
        res['eodd_var'] = np.std(l['eodd_disparity'])
        res['eodd_list'] = l['eodd_disparity']
        return res

    res_train = get_res(train)
    res_val = get_res(val)
    res_test = get_res(test)
    print('Training finished for all seeds.')
    
    return res_train, res_val, res_test

def kde_tradeoff(dataset, hp_test, seeds):
    hp = hp_test.copy()
    result = []
    result_test = []
    for i in hp_test['learning_rate']:
        for k in hp_test['lambda_']:
            for seed in seeds:
                c = []
                c_test = []
                hp['learning_rate'] = i
                hp['lambda_'] = k
                train, val, test = lr_kde_model_runner(dataset, hp, seeds=[seed])
                c.append(hp['learning_rate'])
                c.append(hp['lambda_'])
                c.append(train['accuracy_mean'])
                c.append(val['accuracy_mean'])
                c.append(val['ei_mean'])
                c.append(val['dp_mean'])
                c.append(val['eo_mean'])
                c.append(val['eodd_mean'])
                c_test.append(test['accuracy_mean'])
                c_test.append(test['ei_mean'])
                result_test.append(c_test)
                result.append(c)
    print(tabulate(result, headers=['learning_rate', 'lambda_', 'accuracy_train', 'accuracy_val','ei', 'dp', 'eo', 'eodd']))
    return result, result_test

In [ ]:
EI_hp_test = {}
EI_hp_test['lambda_'] = [0]
EI_hp_test['n_epochs'] = 100
EI_hp_test['batch_size'] = 256
EI_hp_test['fairness'] = ''
EI_hp_test['h'] = 0.01
EI_hp_test['delta_huber'] = 0.5
EI_hp_test['delta_effort'] = 1.1
EI_hp_test['learning_rate'] = [0.01]
EI_hp_test['lambda_'] = np.linspace(0,0.5,20)
EI_hp_test['fairness'] = 'EI'
seeds= np.arange(4)
_, results_kde = kde_tradeoff(dataset, EI_hp_test, seeds)
results_kde_pareto = pareto_frontier_multi(np.squeeze(results_kde))

In [ ]:
results_kde_pareto = pareto_frontier_multi(np.squeeze(results_kde))
results_kde_pareto

In [ ]:
def lr_fb_model_runner(dataset, hp, seeds):
    test = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}
    
    train = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}
    
    val = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}

    def append_res(l,acc,ei,dp,eo,eodd):
        l['accuracy'].append(acc)
        l['ei_disparity'].append(ei)
        l['dp_disparity'].append(dp)
        l['eo_disparity'].append(eo)
        l['eodd_disparity'].append(eodd)

    for i in range(len(seeds)):
        print('training seed', seeds[i] ,'started')
        random.seed(int(seeds[i]))
        np.random.seed(seeds[i])
        torch.manual_seed(seeds[i]) 

        model = logReg(num_features=dataset.XZ_train.shape[1])
        model = model.to(device)
        
        lr = hp['learning_rate']
        optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)
        
        results = trainer_fb_fair(
            model,
            dataset,
            optimizer,
            device,
            n_epochs=hp['n_epochs'],
            batch_size=hp['batch_size'], 
            z_blind=False,
            fairness=hp['fairness'], 
            lambda_=hp['lambda_'], 
            optimal_effort=True, 
            delta_effort=hp['delta_effort']
            )
        
        append_res(train,results.train_acc_hist[-1],results.train_ei_hist[-1],results.train_dp_hist[-1],results.train_eo_hist[-1],results.train_eodd_hist[-1])
        append_res(val,results.val_acc,results.val_ei,results.val_dp,results.val_eo,results.val_eodd)
        append_res(test,results.test_acc,results.test_ei,results.test_dp,results.test_eo,results.test_eodd)

    def get_res(l):
        res = {}
        res['accuracy_mean'] = np.mean(l['accuracy'])
        res['accuracy_var'] = np.std(l['accuracy'])
        res['accuracy_list'] = l['accuracy']
        res['ei_mean'] = np.mean(l['ei_disparity'])
        res['ei_var'] = np.std(l['ei_disparity'])
        res['ei_list'] = l['ei_disparity']
        res['dp_mean'] = np.mean(l['dp_disparity'])
        res['dp_var'] = np.std(l['dp_disparity'])
        res['dp_list'] = l['dp_disparity']
        res['eo_mean'] = np.mean(l['eo_disparity'])
        res['eo_var'] = np.std(l['eo_disparity'])
        res['eo_list'] = l['eo_disparity']
        res['eodd_mean'] = np.mean(l['eodd_disparity'])
        res['eodd_var'] = np.std(l['eodd_disparity'])
        res['eodd_list'] = l['eodd_disparity']
        return res

    res_train = get_res(train)
    res_val = get_res(val)
    res_test = get_res(test)
    print('Training finished for all seeds.')
    
    return res_train, res_val, res_test


def fb_tradeoff(dataset, hp_test, seeds):
    hp = hp_test.copy()
    result = []
    result_test = []
    for i in hp_test['learning_rate']:
        for k in hp_test['lambda_']:
            for seed in seeds:
                c = []
                c_test = []
                hp['learning_rate'] = i
                hp['lambda_'] = k
                train, val, test = lr_fb_model_runner(dataset, hp, seeds=[seed])
                c.append(hp['learning_rate'])
                c.append(hp['lambda_'])
                c.append(train['accuracy_mean'])
                c.append(val['accuracy_mean'])
                c.append(val['ei_mean'])
                c.append(val['dp_mean'])
                c.append(val['eo_mean'])
                c.append(val['eodd_mean'])
                c_test.append(test['accuracy_mean'])
                c_test.append(test['ei_mean'])
                result_test.append(c_test)
                result.append(c)
    print(tabulate(result, headers=['learning_rate', 'lambda_', 'accuracy_train', 'accuracy_val','ei', 'dp', 'eo', 'eodd']))
    return result, result_test

In [ ]:
EI_hp_test = {}
EI_hp_test['learning_rate'] = [0.01]
EI_hp_test['lambda_'] = [0]
EI_hp_test['n_epochs'] = 100
EI_hp_test['batch_size'] = 256
EI_hp_test['delta_effort'] = 1.1
EI_hp_test['lambda_'] = np.linspace(0,0.5,25)
EI_hp_test['fairness'] = 'EI'

_, results_fb = fb_tradeoff(dataset, EI_hp_test, seeds)
results_fb_pareto = pareto_frontier_multi(np.squeeze(results_fb))

In [ ]:
def lr_fc_model_runner(dataset, hp, seeds):
    test = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}
    
    train = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}
    
    val = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}

    def append_res(l,acc,ei,dp,eo,eodd):
        l['accuracy'].append(acc)
        l['ei_disparity'].append(ei)
        l['dp_disparity'].append(dp)
        l['eo_disparity'].append(eo)
        l['eodd_disparity'].append(eodd)

    for i in range(len(seeds)):
        print('training seed', seeds[i] ,'started')
        random.seed(int(seeds[i]))
        np.random.seed(seeds[i])
        torch.manual_seed(seeds[i]) 

        model = logReg(num_features=dataset.XZ_train.shape[1])
        model = model.to(device)
        
        lr = hp['learning_rate']
        optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)
        
        results = trainer_fc_fair(
            model,
            dataset,
            optimizer,
            device,
            n_epochs=hp['n_epochs'],
            batch_size=hp['batch_size'], 
            z_blind=False,
            fairness=hp['fairness'], 
            lambda_=hp['lambda_'],
            optimal_effort=True, 
            delta_effort=hp['delta_effort']
            )
        append_res(train,results.train_acc_hist[-1],results.train_ei_hist[-1],results.train_dp_hist[-1],results.train_eo_hist[-1],results.train_eodd_hist[-1])
        append_res(val,results.val_acc,results.val_ei,results.val_dp,results.val_eo,results.val_eodd)
        append_res(test,results.test_acc,results.test_ei,results.test_dp,results.test_eo,results.test_eodd)

    def get_res(l):
        res = {}
        res['accuracy_mean'] = np.mean(l['accuracy'])
        res['accuracy_var'] = np.std(l['accuracy'])
        res['accuracy_list'] = l['accuracy']
        res['ei_mean'] = np.mean(l['ei_disparity'])
        res['ei_var'] = np.std(l['ei_disparity'])
        res['ei_list'] = l['ei_disparity']
        res['dp_mean'] = np.mean(l['dp_disparity'])
        res['dp_var'] = np.std(l['dp_disparity'])
        res['dp_list'] = l['dp_disparity']
        res['eo_mean'] = np.mean(l['eo_disparity'])
        res['eo_var'] = np.std(l['eo_disparity'])
        res['eo_list'] = l['eo_disparity']
        res['eodd_mean'] = np.mean(l['eodd_disparity'])
        res['eodd_var'] = np.std(l['eodd_disparity'])
        res['eodd_list'] = l['eodd_disparity']
        return res

    res_train = get_res(train)
    res_val = get_res(val)
    res_test = get_res(test)
    print('Training finished for all seeds.')
    
    return res_train, res_val, res_test

def fc_tradeoff(dataset, hp_test, seeds):
    hp = hp_test.copy()
    result = []
    result_test = []
    for i in hp_test['learning_rate']:
        for k in hp_test['lambda_']:
            for seed in seeds:
                c = []
                c_test = []
                hp['learning_rate'] = i
                hp['lambda_'] = k
                train, val, test = lr_fc_model_runner(dataset, hp, seeds=[seed])
                c.append(hp['learning_rate'])
                c.append(hp['lambda_'])
                c.append(train['accuracy_mean'])
                c.append(val['accuracy_mean'])
                c.append(val['ei_mean'])
                c.append(val['dp_mean'])
                c.append(val['eo_mean'])
                c.append(val['eodd_mean'])
                c_test.append(test['accuracy_mean'])
                c_test.append(test['ei_mean'])
                result_test.append(c_test)
                result.append(c)
    print(tabulate(result, headers=['learning_rate', 'lambda_', 'accuracy_train', 'accuracy_val','ei', 'dp', 'eo', 'eodd']))
    return result, result_test

In [ ]:
EI_hp_test = {}
EI_hp_test['learning_rate'] = [0.01]
EI_hp_test['lambda_'] = [0]
EI_hp_test['n_epochs'] = 100
EI_hp_test['batch_size'] = 256
EI_hp_test['delta_effort'] = 1.1
EI_hp_test['lambda_'] = np.linspace(0,1,100)
EI_hp_test['fairness'] = 'EI'

_, results_fc = fc_tradeoff(dataset, EI_hp_test, seeds=[0])
results_fc_pareto = pareto_frontier_multi(np.squeeze(results_fc))

In [ ]:
results_fc_pareto

In [ ]:
np.save('results/german_tradeoff_fb_ei.npy',results_fb_pareto)
np.save('results/german_tradeoff_fc_ei.npy',results_fc_pareto)
np.save('results/german_tradeoff_kde_ei.npy',results_kde_pareto)